# Meta Learning Cache Replacement Policy

## Install Dependency

In [ ]:
# %load extract_tar_gz.py
import argparse
import os


EXTENSION = '.tar.gz'
BLKPARSE = '.blkparse'


def main(input_dir, output_dir):
  assert os.path.isdir(input_dir), "The input directory {} does not exist".format(input_dir)

  input_dir = input_dir.rstrip("/")
  if not os.path.isdir(output_dir):
    print("Creating output directory {}".format(output_dir))
    os.mkdir(output_dir)

  for file in os.listdir(input_dir):
    if not file.endswith(EXTENSION):
      continue
    
    print("Extracting and moving {}".format(file))
    execute_command("tar -xvzf {}".format(input_dir + '/' + file))
    execute_command("mv ./*{} {}".format(BLKPARSE, output_dir))
    

def execute_command(command):
  os.system(command)


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument("--input_dir", type=str, help="the directory containing the tar.gz files", required=True)
  parser.add_argument("--output_dir", type=str, help="the directory containing content of the tar.gz files", required=True)
  args = parser.parse_args() 
  main(args.input_dir, args.output_dir)


In [26]:
import sys
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm 
from collections import Counter, deque, defaultdict
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
import tensorflow as tf


## Decompress tar.gz

In [ ]:
input_dir = "FIU_raw"
output_dir = "FIU_trace"
main(input_dir, output_dir)

## Block Cache Model

### Global Variable

In [27]:
# Maximum block number
maxpos = 1000000000000

# Number of features (Recency, Frequency, Block No.)
num_params = 3

# Cache Size
cache_size = 100


sampling_freq = cache_size

# How many % of cache to use
eviction = int(0.7 * cache_size)  

# Results
lruCorrect = 0
lruIncorrect = 0

lfuCorrect = 0
lfuIncorrect = 0

# Variables
X = np.array([], dtype=np.int64).reshape(0,num_params)
Y = np.array([], dtype=np.int64).reshape(0,1)

In [ ]:
train = "DATA/cheetah.cs.fiu.edu-110108-113008.3.blkparse"

df = pd.read_csv(train, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace)*0.1)]

len(trainBlockTrace)